# 最基本 vector storage index 示例 v0.1

## 初步结论

2024-6-18

- 需要先缩小范围，而不是从全部嵌入中top_k，效果不好
- 默认高级查询差于底层api方式

## 准备

In [1]:
%%time

# 加载llm和embeddings
%run ../utils.py

CPU times: user 3.37 s, sys: 305 ms, total: 3.68 s
Wall time: 3.42 s


## 加载新闻数据

In [11]:
import os
import json

directory = './news'

news_list=[]

# List all files in the specified directory
for file in os.listdir(directory):
    if file.endswith('.json'):
        file_path = os.path.join(directory, file)
        # Open and read the JSON file
        with open(file_path, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)
                news_list.append(data)
            except json.JSONDecodeError as e:
                print(f"Error reading {file_path}: {e}")

len(news_list)

3

In [14]:
news_list[0]

{'title': '中国驻菲律宾使馆发言人就美国军方发动秘密心理战抹黑中国疫苗发表声明',
 'author': '林琛力',
 'publish_time': '2024-06-18 15:34:33',
 'content': '据中国驻菲律宾大使馆微信公众号消息,中国驻菲律宾使馆发言人就美国军方发动秘密心理战抹黑中国疫苗发表声明。\n一、问:近日,英国路透社报道披露,新冠疫情期间美国军方为了遏制中国影响力,发动秘密心理战,使用社交媒体账号假冒菲律宾人散播中国疫苗和抗疫物资不安全的虚假信息,煽动菲民众对中国疫苗的不信任和恐惧,导致菲成为东南亚接种率最低、死亡率最高国家之一,数以万计民众染疫死亡。菲国内出现要求菲政府和国会对此展开调查的呼声。使馆对此有何评论?\n答:中国是负责任大国,新冠疫情期间向各国积极提供疫苗等全球公共产品,是第一个向菲律宾援助抗疫物资和疫苗的国家。中菲两国守望相助,开展抗疫合作,为世界最终战胜疫情做出重要贡献。但中菲抗疫合作经历了很多艰难曲折,其中一个重要原因就是某些域外国家和势力的蓄意阻挠破坏,两国人民对此坚决反对并进行了有力斗争。过去两年,陆续已有一些内幕曝光。上述报告的出台,终于使有些真相天下大白。\n美军的做法令人愤慨,美国的虚伪双标和邪恶本质暴露无遗。美国整天把尊重人权挂在嘴边,但在事关菲律宾人民生命健康这个最大的人权上却恰恰相反,干着草菅人命的事情。美国把对盟友的承诺坚如磐石喊得震天响,但到抗疫这种需要同舟共济的关键时刻却美国优先,让菲律宾人民孤立无援。美国满口仁义道德,实际上不但不帮助菲律宾人民,无视菲律宾民众生命健康,反而出于地缘政治私利千方百计阻止菲律宾人民获得中国援助。在新冠疫苗这样涉及全球公共卫生的重大问题上造谣欺骗,敢冒天下之大不韪。\n人无信不立,国无德不强。如果美国真是自己宣称的样子,就应该停止炮制散布针对别国的虚假信息,切实承担大国应尽的责任,切实尊重地区国家维护民众健康福祉和地区和平稳定的努力。\n二、问:路透社上述报道同时披露,美国军方网上秘密信息行动仍在世界各地继续。菲律宾有评论认为,既然美在疫情期间发起了秘密虚假信息行动来抹黑中国疫苗,就有可能并正在涉海问题上采取同样行动以塑造中国“霸凌”形象。使馆对此有何评论?\n答:我们注意到这些评论。对此,中菲两国人民都应保持高度警惕,擦亮眼睛,及时揭露抵制虚假

## 创建 Document

In [35]:
%%time

from llama_index.core import Document

documents = [Document(text=t['content'], 
                      metadata={"title": t['title'],
                                'publish_time': t['publish_time'],
                                'author': t['author'],
                                'images': t['images'],
                               }) 
             for t in news_list]

for document in documents:
    document.doc_id = document.metadata["title"]

CPU times: user 238 µs, sys: 24 µs, total: 262 µs
Wall time: 244 µs


In [36]:
documents[0]

Document(id_='中国驻菲律宾使馆发言人就美国军方发动秘密心理战抹黑中国疫苗发表声明', embedding=None, metadata={'title': '中国驻菲律宾使馆发言人就美国军方发动秘密心理战抹黑中国疫苗发表声明', 'publish_time': '2024-06-18 15:34:33', 'author': '林琛力', 'images': []}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='据中国驻菲律宾大使馆微信公众号消息,中国驻菲律宾使馆发言人就美国军方发动秘密心理战抹黑中国疫苗发表声明。\n一、问:近日,英国路透社报道披露,新冠疫情期间美国军方为了遏制中国影响力,发动秘密心理战,使用社交媒体账号假冒菲律宾人散播中国疫苗和抗疫物资不安全的虚假信息,煽动菲民众对中国疫苗的不信任和恐惧,导致菲成为东南亚接种率最低、死亡率最高国家之一,数以万计民众染疫死亡。菲国内出现要求菲政府和国会对此展开调查的呼声。使馆对此有何评论?\n答:中国是负责任大国,新冠疫情期间向各国积极提供疫苗等全球公共产品,是第一个向菲律宾援助抗疫物资和疫苗的国家。中菲两国守望相助,开展抗疫合作,为世界最终战胜疫情做出重要贡献。但中菲抗疫合作经历了很多艰难曲折,其中一个重要原因就是某些域外国家和势力的蓄意阻挠破坏,两国人民对此坚决反对并进行了有力斗争。过去两年,陆续已有一些内幕曝光。上述报告的出台,终于使有些真相天下大白。\n美军的做法令人愤慨,美国的虚伪双标和邪恶本质暴露无遗。美国整天把尊重人权挂在嘴边,但在事关菲律宾人民生命健康这个最大的人权上却恰恰相反,干着草菅人命的事情。美国把对盟友的承诺坚如磐石喊得震天响,但到抗疫这种需要同舟共济的关键时刻却美国优先,让菲律宾人民孤立无援。美国满口仁义道德,实际上不但不帮助菲律宾人民,无视菲律宾民众生命健康,反而出于地缘政治私利千方百计阻止菲律宾人民获得中国援助。在新冠疫苗这样涉及全球公共卫生的重大问题上造谣欺骗,敢冒天下之大不韪。\n人无信不立,国无德不强。如果美国真是自己宣称的样子,就应该停止炮制散布针对别国的虚假信息,切实承担大国应尽的责任,切实尊重地区国家维护民众健康福祉和地区和平稳定

In [37]:
documents[0].metadata

{'title': '中国驻菲律宾使馆发言人就美国军方发动秘密心理战抹黑中国疫苗发表声明',
 'publish_time': '2024-06-18 15:34:33',
 'author': '林琛力',
 'images': []}

## 创建和使用默认形式索引

In [40]:
%%time

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)

CPU times: user 86.8 ms, sys: 992 µs, total: 87.8 ms
Wall time: 4.77 s


In [41]:
%%time

query_engine = index.as_query_engine(streaming=True,)
response = query_engine.query("中澳消除了哪些分歧")
response.print_response_stream()

根据提供的上下文信息，文章中并没有明确指出中澳之间具体消除了哪些分歧。文中主要讨论了澳大利亚国防政策的变化以及其外交政策对美国利益的依赖性，并提到了澳大利亚在亚洲尤其是对中国的态度上仍然受到美国行动的影响。因此，可以推测中澳之间的某些战略或政策上的分歧可能正在减少或者调整，但具体的分歧内容并未直接提及。CPU times: user 200 ms, sys: 7.09 ms, total: 207 ms
Wall time: 4.95 s


In [42]:
%%time

query_engine = index.as_query_engine(streaming=True,)
response = query_engine.query("德企在华面临的主要挑战是啥")
response.print_response_stream()

根据提供的上下文信息，文中并未直接提及德企在华面临的具体挑战。因此，无法直接从给定的信息中提取出关于德企在中国主要面临的挑战的详细答案。

然而，通常情况下，跨国企业在进入和运营于中国市场时可能会面临以下一些普遍挑战：

1. **文化差异**：理解并适应中国市场的独特文化和商业习惯。
2. **市场准入与法规**：遵守中国的法律法规、行业标准以及市场准入要求。
3. **供应链管理**：在中国建立或整合供应链，可能需要面对物流成本、运输时间及质量控制等挑战。
4. **人才获取和保留**：吸引并留住具有中国业务所需技能的本地人才。
5. **市场竞争激烈**：中国市场竞争激烈，尤其是在高科技和消费品领域。
6. **知识产权保护**：确保其产品和技术不受侵犯，同时在中国建立有效的知识产权保护机制。
7. **经济环境变化**：中国经济政策、市场波动以及全球经济形势的变化可能影响业务运营。

请注意，这些挑战是基于一般跨国企业进入新兴市场的经验总结，并不一定完全适用于所有德企在华的具体情况。具体到德企的挑战可能会根据其行业特性、战略定位和市场策略有所不同。CPU times: user 357 ms, sys: 17.5 ms, total: 374 ms
Wall time: 6.39 s


In [43]:
%%time

query_engine = index.as_query_engine(streaming=True,)
response = query_engine.query("美国军方发动秘密心理战主要做了啥")
response.print_response_stream()

美国军方为了遏制中国影响力，采取了秘密心理战行动，其主要做法包括在疫情期间利用社交媒体账号冒充菲律宾人传播关于中国疫苗和抗疫物资的虚假信息。这些不实信息旨在煽动菲律宾民众对中国疫苗的安全性和有效性产生怀疑与恐惧情绪。这一行为导致菲律宾成为东南亚地区接种率最低、死亡率最高的国家之一，大量民众因此染疫死亡。这一行动引发了菲国内对政府及国会要求展开调查的声音。CPU times: user 130 ms, sys: 22.9 ms, total: 153 ms
Wall time: 2.92 s


## 使用qdrant向量存储索引

### 创建/加载qdrant向量存储索引

In [50]:
%%time

import qdrant_client
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client.models import Distance, VectorParams
from llama_index.core import StorageContext

client = qdrant_client.QdrantClient(
    # location=":memory:",
    host="ape", port=6333,
)

collection_name = "news"
vector_store = QdrantVectorStore(client=client, collection_name=collection_name)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index=None

collections = client.get_collections()

# 检查集合是否存在
if any(collection.name == collection_name for collection in collections.collections):
    print(f"Collection '{collection_name}' exists.")
    index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
else:
    print(f"Collection '{collection_name}' does not exist.")

    index = VectorStoreIndex.from_documents(
        documents,
        storage_context=storage_context,
    )
    

Collection 'news' exists.
CPU times: user 51.4 ms, sys: 0 ns, total: 51.4 ms
Wall time: 52.7 ms


### 高级查询

In [51]:
%%time

query_engine = index.as_query_engine(streaming=True,)
response = query_engine.query("中澳消除了哪些分歧")
response.print_response_stream()

根据提供的上下文信息，没有直接提到中澳之间具体消除了哪些分歧。不过，可以推测文章可能讨论了中澳在某些方面存在的一些分歧和紧张关系，并提到了澳大利亚对中国的态度与美国的紧密关联。因此，中澳之间的主要分歧可能包括贸易、安全政策、地缘政治影响力等方面的问题。

然而，由于没有直接引用具体段落或详细信息来明确指出哪些分歧被消除，我们无法提供具体的答案。通常，在这样的讨论中，“消除分歧”可能意味着双方在某些领域寻求共识、改善沟通或者调整策略以减少误解和紧张关系。但具体到中澳关系的上下文，这需要更详细的分析和对文章内容的深入理解才能给出准确的答案。CPU times: user 222 ms, sys: 15.8 ms, total: 237 ms
Wall time: 6.4 s


### 底层api定制查询

In [53]:
%%time

from llama_index.core import get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(streaming=True,)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.3)],
)

response = query_engine.query("中澳消除了哪些分歧")
response.print_response_stream()

在过去12个月里，中澳之间的经济关系出现了积极转变，一系列反贸易措施正在逐步取消。尽管在外交关系上存在紧张，两国的贸易仍在继续扩大，并且更多地受到资源互补性的驱动而非外交多变性的影响。这表明，在经济层面上，双方正努力消除一些分歧。

然而，澳大利亚仍难以摆脱以下两个因素的束缚：
1. 历史上的焦虑：澳大利亚对中国和亚洲的态度中存在一种长期的焦虑感。
2. 对大国抛弃的恐惧：澳大利亚人内心深处有一种被大国（可能指的是中国）抛弃的担忧。

这些历史根源和社会心理因素表明，在外交关系正常化的同时，澳大利亚在处理与中国的经济联系时仍面临内部的矛盾态度。一方面希望从两国关系中获益，另一方面对中国持有消极和警惕的态度。这种长期存在的矛盾反映了澳大利亚社会对与中国交往存在国家安全风险的认识，并支持采取更强硬立场的观点。

因此，虽然在经济层面上双方可能在努力消除一些分歧，但在外交政策、公众情绪和社会心理层面，中澳之间仍存在复杂的分歧需要解决。CPU times: user 606 ms, sys: 25.9 ms, total: 632 ms
Wall time: 23 s


In [58]:
len(response.source_nodes)

10

In [61]:
response.source_nodes[0]

NodeWithScore(node=TextNode(id_='390956fb-1444-4527-a119-038784854599', embedding=None, metadata={'title': '鲍韶山:中澳正在消除分歧,但澳大利亚仍是一个焦虑的国家', 'publish_time': '2024-06-17 07:43:21', 'author': '苏堤', 'images': ['https://i.guancha.cn/news/external/2024/06/14/20240614222611379.png', 'https://i.guancha.cn/news/mainland/2024/06/14/20240614223736493.png', 'https://i.guancha.cn/news/official/2024/06/14/20240614224316558.png', 'https://i.guancha.cn/news/mainland/2024/06/14/20240614230242540.jpg', 'https://i.guancha.cn/news/social/2024/06/14/20240614231040450.png', 'https://i.guancha.cn/news/official/2024/06/14/20240614235215709.png', 'https://i.guancha.cn/shiping-banner.jpg']}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='鲍韶山:中澳正在消除分歧,但澳大利亚仍是一个焦虑的国家', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'title': '鲍韶山:中澳正在消除分歧,但澳大利亚仍是一个焦虑的国家', 'publish_time': '2024-06-17 07:43:21', 'author': '苏堤', 'images': ['https:

In [60]:
response.source_nodes[0].score

0.49106956

In [62]:
response.source_nodes[0].metadata['title']

'鲍韶山:中澳正在消除分歧,但澳大利亚仍是一个焦虑的国家'

In [63]:
response.source_nodes[1].metadata['title']

'中国驻菲律宾使馆发言人就美国军方发动秘密心理战抹黑中国疫苗发表声明'

In [64]:
response.source_nodes[2].metadata['title']

'鲍韶山:中澳正在消除分歧,但澳大利亚仍是一个焦虑的国家'

In [65]:
# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    # similarity_top_k=10,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(streaming=True,)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.3)],
)

response = query_engine.query("中澳消除了哪些分歧")
response.print_response_stream()

根据提供的上下文信息，没有直接提到中澳之间具体消除了哪些分歧。文章主要讨论了澳大利亚国防政策的变化以及其外交政策对美国利益的依赖性，并提到了澳大利亚在亚洲尤其是对中国的态度上与美国紧密相关的情况。因此，可以推测中澳之间的某些分歧可能包括：

1. **国防自主性和主权行动能力**：过去20年中，澳大利亚的国防自主和主权行动能力被认为已经丧失，这表明两国在国家安全和军事策略上的观点可能存在差异。

2. **外交政策与决策依赖性**：文章提到澳大利亚的外交政策更多取决于美国如何应对地区霸权地位的逐渐丧失，暗示了澳大利亚在某些国际事务上可能对美国有过度依赖的情况，这可能是中澳之间的一个分歧点。

3. **亚洲和中国政策**：文章指出澳大利亚对整个亚洲尤其是对中国的态度与华盛顿（即美国）的一举一动密切相关。这意味着两国在处理与中国的关系时可能存在策略或立场上的差异。

综上所述，中澳之间的具体分歧可能涉及国防自主、外交决策的依赖性以及在亚洲特别是与中国关系的处理方式等方面存在不同观点和策略选择。然而，具体的分歧细节并未在提供的上下文中明确给出。

In [67]:
# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=1,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(streaming=True,)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.3)],
)

response = query_engine.query("中澳消除了哪些分歧")
response.print_response_stream()

根据提供的上下文信息，文中并没有明确指出中澳之间具体消除了哪些分歧。文章主要讨论了澳大利亚国防政策的变化以及其外交和贸易政策在很大程度上依赖于美国的状况，并提到了澳大利亚对亚洲尤其是对中国的态度与美国的一致性。因此，无法从给定的信息中直接得出中澳之间消除了哪些具体的分歧。

In [69]:
# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=5,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(streaming=True,)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.3)],
)

response = query_engine.query("中澳消除了哪些分歧")
response.print_response_stream()

在过去的12个月里，中澳之间在经济领域展现出了积极的变化，一系列限制措施开始解除，促进了双边贸易的增长。尽管外交关系上仍存有紧张，但两国间的贸易活动更多地基于资源互补性而非外交政策的多变性，这表明在经济层面上双方已消除了部分分歧。

然而，澳大利亚在面对中澳关系时仍然受到两个关键因素的影响：
1. 历史性的焦虑：澳大利亚对中国和亚洲的态度中存在长期存在的不安情绪。
2. 对大国抛弃的恐惧：公众内心深处存在着被大国（可能指的是中国）忽视或抛弃的可能性。

尽管经济层面上的紧张有所缓解，但在政治与安全领域，两国之间的分歧并未完全消除。公众对于中国的看法显示出对“咄咄逼人”的感知增加以及对中国潜在入侵的担忧，这些因素反映了未解决的分歧仍然存在。